### This code imports Landsat 8 images from AWS server

In [3]:
import pandas as pd
import geopandas as gpd
import os, shutil
from glob import glob
import numpy as np
import xarray as xr
import scipy.ndimage
from matplotlib import pyplot as plt
%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'

In [2]:
import shapely
import datetime

Set path for images:

In [1]:
cd

/home/jovyan


In [2]:
WRS_PATH = './L8_data/external/Landsat8/WRS2_descending_0.zip'
LANDSAT_PATH = os.path.dirname(WRS_PATH)

NameError: name 'os' is not defined

Only neeeds to be done once: import the geometry csv for Landsat 8 row and path boundaries


!wget -P {LANDSAT_PATH} https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip



In [5]:
shutil.unpack_archive(WRS_PATH, os.path.join(LANDSAT_PATH, 'wrs2')) 

Convert the data into a geopandas dataset:

In [6]:
wrs = gpd.GeoDataFrame.from_file('L8_data/external/Landsat8/wrs2/WRS2_descending.shp')

Import the grounding lines for Antarctic iceshelves from https://nsidc.org/data/nsidc-0709:

In [7]:
ice_shelves = gpd.read_file('REMAWaterRouting/IceShelf_Antarctica_v02/IceShelf_Antarctica_v02.shp')

In [8]:
amery = ice_shelves[ice_shelves['NAME']=='Amery'] #Here we select the Amery ice shelf

In [9]:
amery = amery.to_crs('EPSG:4326')


Using the intersects function, we find what row/path overlaps with our area of interest:

In [10]:
wrs_intersection  = wrs[wrs.intersects(shapely.geometry.Polygon.from_bounds(float(amery.bounds.minx),
                                                    float(amery.bounds.miny),float(amery.bounds.maxx),float(amery.bounds.maxy)))]

In [11]:
paths, rows = wrs_intersection['PATH'].values, wrs_intersection['ROW'].values

In [12]:
paths

array([125, 125, 125, 125, 132, 132, 132, 123, 123, 123, 130, 130, 130,
       130, 121, 128, 128, 128, 128, 128, 126, 126, 126, 126, 126, 133,
       133, 124, 124, 124, 131, 131, 131, 122, 122, 129, 129, 129, 129,
       129, 127, 127, 127, 127, 127])

In [5]:
#Import the list of available imagery on the amazon aws server:
s3_scenes = pd.read_csv('http://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz', compression='gzip')


In [14]:
s3_scenes.head(3)


,productId,entityId,acquisitionDate,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url
0,LC08_L1TP_149039_20170411_20170415_01_T1,LC81490392017101LGN00,2017-04-11 05:36:29.349932,0.00,L1TP,149,39,29.22165,72.41205,31.34742,74.84666,https://s3-us-west-2.amazonaws.com/landsat-pds...
1,LC08_L1TP_012001_20170411_20170415_01_T1,LC80120012017101LGN00,2017-04-11 15:14:40.001201,0.15,L1TP,12,1,79.51504,-22.06995,81.90314,-7.44339,https://s3-us-west-2.amazonaws.com/landsat-pds...
2,LC08_L1TP_012002_20170411_20170415_01_T1,LC80120022017101LGN00,2017-04-11 15:15:03.871058,0.38,L1TP,12,2,78.74882,-29.24387,81.14549,-15.04330,https://s3-us-west-2.amazonaws.com/landsat-pds...


In [15]:
b = (paths > 126) & (paths < 128)
paths = paths[b]
rows = rows[b]

In [16]:
# Empty list to add the images
bulk_list = []

# Iterate through paths and rows
for path, row in zip(paths, rows):

    print('Path:',path, 'Row:', row)

    # Filter the Landsat Amazon S3 table for images matching path, row, cloudcover and processing state.
    scenes = s3_scenes[(s3_scenes.path == path) & (s3_scenes.row == row) & 
                       (s3_scenes.cloudCover <= 5)] #cloud cover less than 5%
    scenes = scenes[scenes.acquisitionDate.str.slice(5,7).astype(int)<2] #acquired in January
    print(' Found {} images\n'.format(len(scenes)))

    # If any scenes exists, select the one that have the minimum cloudCover.
    if len(scenes):
        scene = scenes.sort_values('cloudCover').iloc[0]
    else:
        continue
    # Add the selected scene to the bulk download list.
    bulk_list.append(scene)

Path: 127 Row: 108
 Found 0 images

Path: 127 Row: 109
 Found 4 images

Path: 127 Row: 110
 Found 5 images

Path: 127 Row: 111
 Found 5 images

Path: 127 Row: 112
 Found 5 images



In [17]:
bulk_frame = pd.concat(bulk_list, 1).T
bulk_frame

,productId,entityId,acquisitionDate,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url
349640,LC08_L1GT_127109_20180114_20180114_01_RT,LC81271092018014LGN00,2018-01-14 03:49:00.905936,0,L1GT,127,109,-70.8375,69.5321,-68.9107,76.6728,https://s3-us-west-2.amazonaws.com/landsat-pds...
349641,LC08_L1GT_127110_20180114_20180114_01_RT,LC81271102018014LGN00,2018-01-14 03:49:24.881695,0,L1GT,127,110,-72.1631,67.3456,-70.2626,75.0568,https://s3-us-west-2.amazonaws.com/landsat-pds...
1834148,LC08_L1GT_127111_20200120_20200120_01_RT,LC81271112020020LGN00,2020-01-20 03:49:54.470726,0.04,L1GT,127,111,-73.4731,64.8826,-71.5983,73.2477,https://s3-us-west-2.amazonaws.com/landsat-pds...
1834149,LC08_L1GT_127112_20200120_20200120_01_RT,LC81271122020020LGN00,2020-01-20 03:50:18.442248,0.14,L1GT,127,112,-74.7588,62.0524,-72.9162,71.1638,https://s3-us-west-2.amazonaws.com/landsat-pds...


In [ ]:
conda install bs4

In [ ]:
# Import requests and beautiful soup
import requests
from bs4 import BeautifulSoup

# For each row
for i, row in bulk_frame.iterrows():

    # Print some the product ID
    print('\n', 'EntityId:', row.productId, '\n')
    print(' Checking content: ', '\n')

    # Request the html text of the download_url from the amazon server. 
    # download_url example: https://landsat-pds.s3.amazonaws.com/c1/L8/139/045/LC08_L1TP_139045_20170304_20170316_01_T1/index.html
    response = requests.get(row.download_url)

    # If the response status code is fine (200)
    if response.status_code == 200:

        # Import the html to beautiful soup
        html = BeautifulSoup(response.content, 'html.parser')

        # Create the dir where we will put this image files.
        entity_dir = os.path.join(LANDSAT_PATH, row.productId)
        os.makedirs(entity_dir, exist_ok=True)

        # Second loop: for each band of this image that we find using the html <li> tag
        for li in html.find_all('li'):

            # Get the href tag
            file = li.find_next('a').get('href')

            print('  Downloading: {}'.format(file))

            # Download the files
            # code from: https://stackoverflow.com/a/18043472/5361345

            response = requests.get(row.download_url.replace('index.html', file), stream=True)

            with open(os.path.join(entity_dir, file), 'wb') as output:
                shutil.copyfileobj(response.raw, output)
            del response

### Using the lake detection algorithm of Moussavi et al. (2020), we iterate through the Landsat images and extract lake boundaries.

In [ ]:
LakeMasks = {}
pi = np.pi
for df_index, row in bulk_frame.head().iterrows():
    mtl_path = str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_MTL.txt')
    mtl_var = {}
    with open(mtl_path) as MTL:
        for line in MTL:
            name, var = line.partition("=")[::2]
            mtl_var[name.strip()] = var

    B2 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B2.TIF'))
    B2 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_2']))+float(mtl_var['REFLECTANCE_ADD_BAND_2'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180)) * 10000; #DN to TOA reflectance conversion

    B3 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B3.TIF'))
    B3 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_3']))+float(mtl_var['REFLECTANCE_ADD_BAND_3'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion

    B4 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B4.TIF'))
    B4 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_4']))+float(mtl_var['REFLECTANCE_ADD_BAND_4'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion

    B6 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B6.TIF'))
    B6 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_6']))+float(mtl_var['REFLECTANCE_ADD_BAND_6'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion

    B10 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B10.TIF'))
    B10 = ((B2*float(mtl_var['RADIANCE_MULT_BAND_10']))+float(mtl_var['RADIANCE_ADD_BAND_10'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion
    B10=  float(mtl_var['K2_CONSTANT_BAND_10'])/np.log((float(mtl_var['K1_CONSTANT_BAND_10'])/B10)+1) * 10;

    # Calculation of NDWI
    NDWI = (B2-B4)/(B2+B4); # This is NDWI
    NDWI = NDWI*1000;


    # Calculation of NDSI

    NDSI = (B3-B6)/(B3+B6); # This is NDSI
    NDSI = NDSI*1000;


    # Calculation of TIRS/Blue

    TIRS_Blue= B10/B2; #This is TIRS/Blue 
    TIRS_Blue = TIRS_Blue*1000;


    # Rock outcrop/Seawater Masking
    rock_mask = np.zeros(B4.shape);
    rock_mask =np.int_(rock_mask);
    index = np.bitwise_and(np.bitwise_and(TIRS_Blue > 6500, B4 > 0), B2 <3500)
    rock_mask[index.values]=1;  #B4 filtering --> to avoid including image borders

    # Cloud Masking
    cloud_mask = np.zeros(B4.shape);
    cloud_mask = np.int_(cloud_mask);
    index = np.bitwise_and(np.bitwise_and(B6 > 1000 , NDSI < 8000),np.bitwise_and(B2 >6000 , B2 < 9500))
    cloud_mask[index.values] = 1;


    # Lake Masking
    lake_mask = np.zeros(B4.shape);
    lake_mask = np.int_(lake_mask);
    index = np.bitwise_and(np.bitwise_and(NDWI > 1900 ,(B3-B4)> 700), (B2-B3)>700)
    lake_mask[index.values] = 1; 
    lake_mask[rock_mask==1]=0;
    lake_mask[cloud_mask==1]=0;


    #writing to xarray
    LakeMasks[df_index] = lake_mask

In [18]:
df_index = 349641
pi=np.pi
mtl_path = str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_MTL.txt')
mtl_var = {}
with open(mtl_path) as MTL:
    for line in MTL:
        name, var = line.partition("=")[::2]
        mtl_var[name.strip()] = var

B2 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B2.TIF'))
B2 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_2']))+float(mtl_var['REFLECTANCE_ADD_BAND_2'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180)) * 10000; #DN to TOA reflectance conversion

B3 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B3.TIF'))
B3 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_3']))+float(mtl_var['REFLECTANCE_ADD_BAND_3'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion

B4 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B4.TIF'))
B4 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_4']))+float(mtl_var['REFLECTANCE_ADD_BAND_4'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion

B6 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B6.TIF'))
B6 = ((B2*float(mtl_var['REFLECTANCE_MULT_BAND_6']))+float(mtl_var['REFLECTANCE_ADD_BAND_6'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion

B10 = xr.open_rasterio(str('L8_data/external/Landsat8/'+bulk_frame.productId[df_index]+'/'+bulk_frame.productId[df_index]+'_B10.TIF'))
B10 = ((B2*float(mtl_var['RADIANCE_MULT_BAND_10']))+float(mtl_var['RADIANCE_ADD_BAND_10'])/np.sin(float(mtl_var['SUN_ELEVATION'])* pi / 180))* 10000; #DN to TOA reflectance conversion
B10=  float(mtl_var['K2_CONSTANT_BAND_10'])/np.log((float(mtl_var['K1_CONSTANT_BAND_10'])/B10)+1) * 10;

# Calculation of NDWI
NDWI = (B2-B4)/(B2+B4); # This is NDWI
NDWI = NDWI*1000;


# Calculation of NDSI

NDSI = (B3-B6)/(B3+B6); # This is NDSI
NDSI = NDSI*1000;


# Calculation of TIRS/Blue

TIRS_Blue= B10/B2; #This is TIRS/Blue 
TIRS_Blue = TIRS_Blue*1000;


# Rock outcrop/Seawater Masking
rock_mask = np.zeros(B4.shape);
rock_mask =np.int_(rock_mask);
index = np.bitwise_and(np.bitwise_and(TIRS_Blue > 6500, B4 > 0), B2 <3500)
rock_mask[index.values]=1;  #B4 filtering --> to avoid including image borders

# Cloud Masking
cloud_mask = np.zeros(B4.shape);
cloud_mask = np.int_(cloud_mask);
index = np.bitwise_and(np.bitwise_and(B6 > 1000 , NDSI < 8000),np.bitwise_and(B2 >6000 , B2 < 9500))
cloud_mask[index.values] = 1;


# Lake Masking
lake_mask = np.zeros(B4.shape);
lake_mask = np.int_(lake_mask);
index = np.bitwise_and(np.bitwise_and(NDWI > 1900 ,(B3-B4)> 700), (B2-B3)>700)
lake_mask[index.values] = 1; 
lake_mask[rock_mask==1]=0;
lake_mask[cloud_mask==1]=0;